In [ ]:
import pandas as pd
import numpy as np
import re
import os
from scipy.stats import spearmanr
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# -----------------------------
# CONFIGURATION
# -----------------------------
CCLE_DRUG_PATH = "./data/ccle_drug_common_with_tcga.csv"
CCLE_EXPR_PATH = "./data/OmicsExpressionTPMLogp1HumanProteinCodingGenes.csv"
TCGA_EXPR_PATH = "./data/TCGA_RNAseq_TPM.csv"
TCGA_ANNO_PATH = "./data/tcga_drug_common_with_ccle.csv"
ONCOKB_PATH    = "./data/cancerGeneList.tsv"

OUTDIR = "./tcga_drug_sensitivity_v2_all_drugs"
os.makedirs(OUTDIR, exist_ok=True)

STRATEGIES = [
    {"name": "quantile_30_oncokb_all", "auc_q": 0.3, "use_spearman": False, "gene_filter": "oncokb_all"},
    {"name": "quantile_20_protein", "auc_q": 0.2, "use_spearman": False, "gene_filter": "protein_coding"},
    {"name": "spearman_top20_oncokb", "auc_q": 0.5, "use_spearman": True,  "gene_filter": "oncokb_only"},
]

MIN_CELL_LINES = 100
MIN_SAMPLES_PER_GROUP = 50
MIN_VALID_GENES = 5

# -----------------------------
# UTILITIES
# -----------------------------
def clean_gene(g):
    return re.sub(r"\s*\(.*?\)", "", str(g)).upper().strip()

def norm_drug(x):
    return re.sub(r"[^A-Z0-9]", "", str(x).upper())

def compute_singscore_v2(expr, up_genes, down_genes=None):
    up_genes = [g for g in up_genes if g in expr.columns]
    if down_genes is not None:
        down_genes = [g for g in down_genes if g in expr.columns]

    if len(up_genes) < 3 and (down_genes is None or len(down_genes) < 3):
        return pd.Series(index=expr.index, dtype=float)

    n_total = expr.shape[1]
    ranks = expr.rank(axis=1, method="average", pct=True)

    def normalize(score, n):
        min_r = (1 + n) / (2 * n_total)
        max_r = 1 - min_r
        return 2 * (score - min_r) / (max_r - min_r) - 1

    up_score = normalize(ranks[up_genes].mean(axis=1), len(up_genes)) if len(up_genes) >= 3 else None
    down_score = normalize((1 - ranks[down_genes]).mean(axis=1), len(down_genes)) if down_genes and len(down_genes) >= 3 else None

    if up_score is not None and down_score is not None:
        raw = up_score - down_score
        return raw - raw.median()
    return up_score if up_score is not None else down_score

def select_primary_sample(barcodes):
    barcodes = barcodes.astype(str)
    patients = barcodes.str.slice(0, 12)
    df = pd.DataFrame({"sample": barcodes, "patient": patients})
    df["sample_type"] = barcodes.str.slice(13, 15)

    primary = df[df["sample_type"] == "01"].drop_duplicates("patient")
    other = df[~df["patient"].isin(primary["patient"])].drop_duplicates("patient")

    result = pd.concat([primary, other]).set_index("patient")["sample"].to_dict()
    print(f"Primary samples selected: {len(result)} patients")
    return result

# -----------------------------
# LOAD DATA
# -----------------------------
print("STEP 1: Loading data")

ccle_drug = pd.read_csv(CCLE_DRUG_PATH)
ccle_exp  = pd.read_csv(CCLE_EXPR_PATH)
tcga      = pd.read_csv(TCGA_EXPR_PATH)
anno      = pd.read_csv(TCGA_ANNO_PATH)
oncokb    = pd.read_csv(ONCOKB_PATH, sep="\t")

# -----------------------------
# ONCOKB GENE SETS
# -----------------------------
if "Hugo Symbol" in oncokb.columns:
    oncogenes_all = set(oncokb["Hugo Symbol"].astype(str).str.upper())
elif "Gene Symbol" in oncokb.columns:
    oncogenes_all = set(oncokb["Gene Symbol"].astype(str).str.upper())
else:
    raise ValueError("Gene symbol column not found in OncoKB")

if "Gene Type" in oncokb.columns:
    oncogenes_only = set(
        oncokb[oncokb["Gene Type"].str.upper().isin(["ONCOGENE", "ONCO"])]
        ["Hugo Symbol"].astype(str).str.upper()
    )
elif "Is Oncogene" in oncokb.columns:
    oncogenes_only = set(
        oncokb[oncokb["Is Oncogene"] == "Yes"]["Hugo Symbol"].astype(str).str.upper()
    )
else:
    oncogenes_only = oncogenes_all

# -----------------------------
# CCLE EXPRESSION
# -----------------------------
print("STEP 2: Processing CCLE expression")

common_models = ccle_drug.iloc[:, 0].astype(str)
expr_base = ccle_exp[ccle_exp.ModelID.isin(common_models)].set_index("ModelID")

meta_cols = [
    "Unnamed: 0", "SequencingID", "IsDefaultEntryForModel",
    "ModelConditionID", "IsDefaultEntryForMC"
]
expr_base = expr_base.drop(columns=[c for c in meta_cols if c in expr_base.columns], errors="ignore")

expr_base = expr_base.apply(pd.to_numeric, errors="coerce")
expr_base = expr_base[~expr_base.index.duplicated()]
expr_base.columns = [clean_gene(g) for g in expr_base.columns]
expr_base = expr_base.groupby(expr_base.columns, axis=1).mean()

expr_oncokb_all  = expr_base[[g for g in expr_base.columns if g in oncogenes_all]]
expr_oncokb_only = expr_base[[g for g in expr_base.columns if g in oncogenes_only]]
expr_protein    = expr_base

# -----------------------------
# TCGA EXPRESSION
# -----------------------------
print("STEP 3: Processing TCGA expression")

sid_col = "sample_id" if "sample_id" in tcga.columns else [c for c in tcga.columns if str(c).startswith("TCGA-")][0]
tcga = tcga.set_index(sid_col).apply(pd.to_numeric, errors="coerce")
tcga = tcga.dropna(axis=1, how="all")

tcga_log = np.log2(tcga + 1.0)
tcga_log.columns = [clean_gene(g) for g in tcga_log.columns]
tcga_log = tcga_log.groupby(tcga_log.columns, axis=1).mean()

tcga_oncokb_all  = tcga_log[[g for g in tcga_log.columns if g in oncogenes_all]]
tcga_oncokb_only = tcga_log[[g for g in tcga_log.columns if g in oncogenes_only]]
tcga_protein    = tcga_log

patient_to_sample = select_primary_sample(tcga_log.index.to_series())

# -----------------------------
# ANNOTATION
# -----------------------------
print("STEP 4: Processing annotation")

anno["patient_id"] = anno["bcr_patient_barcode"].astype(str)
anno["drug_norm"] = anno["drug_name"].apply(norm_drug)

ccle_drug_map = {}
for col in ccle_drug.columns[1:]:
    key = norm_drug(col.split("(")[0])
    if key not in ccle_drug_map:
        ccle_drug_map[key] = col

common_drugs = set(anno["drug_norm"]) & set(ccle_drug_map)

# -----------------------------
# MAIN LOOP
# -----------------------------
print("STEP 5: Running strategies")

all_records = []

for strategy in STRATEGIES:
    print(f"Strategy: {strategy['name']}")

    if strategy["gene_filter"] == "oncokb_all":
        expr = expr_oncokb_all
        tcga_expr = tcga_oncokb_all
    elif strategy["gene_filter"] == "oncokb_only":
        expr = expr_oncokb_only
        tcga_expr = tcga_oncokb_only
    else:
        expr = expr_protein
        tcga_expr = tcga_protein

    records = []

    for drug_norm in tqdm(sorted(common_drugs)):
        if drug_norm not in ccle_drug_map:
            continue

        drug_ccle = ccle_drug_map[drug_norm]
        auc = ccle_drug.set_index(ccle_drug.columns[0])[drug_ccle].astype(float)
        common = expr.index.intersection(auc.index)

        if len(common) < MIN_CELL_LINES:
            continue

        expr_sub = expr.loc[common]
        auc_sub  = auc.loc[common]

        q_low  = auc_sub.quantile(strategy["auc_q"])
        q_high = auc_sub.quantile(1 - strategy["auc_q"])

        sens_idx = auc_sub[auc_sub <= q_low].index
        res_idx  = auc_sub[auc_sub >= q_high].index

        if len(sens_idx) < MIN_SAMPLES_PER_GROUP or len(res_idx) < MIN_SAMPLES_PER_GROUP:
            continue

        sens = expr_sub.loc[sens_idx]
        res  = expr_sub.loc[res_idx]

        if strategy["use_spearman"]:
            corrs = []
            for g in expr_sub.columns:
                r, _ = spearmanr(auc_sub, expr_sub[g], nan_policy="omit")
                if not np.isnan(r):
                    corrs.append((g, r))
            if len(corrs) < MIN_VALID_GENES:
                continue

            corrs_pos = sorted([x for x in corrs if x[1] > 0], key=lambda x: abs(x[1]), reverse=True)
            corrs_neg = sorted([x for x in corrs if x[1] < 0], key=lambda x: abs(x[1]), reverse=True)

            up_sensitive = [g for g, _ in corrs_pos[:25]]
            up_resistant = [g for g, _ in corrs_neg[:25]]
        else:
            diff = sens.mean() - res.mean()
            top_n = max(10, min(50, int(len(diff.dropna()) * 0.1)))
            up_sensitive = diff.nlargest(top_n).index.tolist()
            up_resistant = diff.nsmallest(top_n).index.tolist()

        if len(up_sensitive) < 5 or len(up_resistant) < 5:
            continue

        scores = compute_singscore_v2(tcga_expr, up_sensitive, up_resistant)

        for _, r in anno[anno["drug_norm"] == drug_norm].iterrows():
            pid = r["patient_id"]
            if pid not in patient_to_sample:
                continue
            sid = patient_to_sample[pid]
            if sid not in scores.index or pd.isna(scores.loc[sid]):
                continue

            records.append({
                "patient_id": pid,
                "sample_id": sid,
                "drug_name": r["drug_name"],
                "cancer": r.get("Cancer", np.nan),
                "clinical_response": r.get("measure_of_response", np.nan),
                "predicted_sensitivity_score": float(scores.loc[sid]),
                "strategy": strategy["name"],
                "n_genes_up": len(up_sensitive),
                "n_genes_down": len(up_resistant),
            })

    out_file = f"{OUTDIR}/{strategy['name']}.tsv"
    pd.DataFrame(records).to_csv(out_file, sep="\t", index=False)
    all_records.extend(records)

# -----------------------------
# INTEGRATION
# -----------------------------
print("STEP 6: Integrating strategies")

final_df = pd.DataFrame(all_records)

integrated = final_df.groupby(
    ["patient_id", "drug_name", "cancer", "clinical_response"]
).agg(
    predicted_sensitivity_score_integrated=("predicted_sensitivity_score", "mean"),
    n_strategies=("strategy", "count"),
    avg_n_genes_up=("n_genes_up", "mean"),
    avg_n_genes_down=("n_genes_down", "mean"),
).reset_index()

integrated.to_csv(f"{OUTDIR}/INTEGRATED_all_strategies.tsv", sep="\t", index=False)
integrated[integrated["n_strategies"] >= 2].to_csv(
    f"{OUTDIR}/INTEGRATED_multi_strategy_only.tsv", sep="\t", index=False
)

print("Pipeline completed")

STEP 1: 載入資料
✓ CCLE 藥物資料: (840, 32)
✓ CCLE 表達資料: (1754, 19221)
✓ TCGA 表達資料: (1228, 59432)
✓ TCGA 註釋資料: (1286, 15)
✓ OncoKB 資料: (1216, 16)

=== OncoKB 檔案資訊 ===
欄位: ['Hugo Symbol', 'Entrez Gene ID', 'GRCh37 Isoform', 'GRCh37 RefSeq', 'GRCh38 Isoform', 'GRCh38 RefSeq', 'Gene Type', '# of occurrence within resources (Column J-P)', 'OncoKB Annotated', 'MSK-IMPACT', 'MSK-HEME', 'FOUNDATION ONE', 'FOUNDATION ONE HEME', 'Vogelstein', 'COSMIC CGC (v99)', 'Gene Aliases']

Gene Type 分布:
Gene Type
ONCOGENE                 422
TSG                      331
INSUFFICIENT_EVIDENCE    136
ONCOGENE_AND_TSG          72
NEITHER                   17
Name: count, dtype: int64

✓ OncoKB 全基因: 1216
✓ OncoKB oncogenes only: 422

STEP 2: 處理 CCLE 表達資料
✓ CCLE 總基因數: 19215
✓ CCLE oncokb_all: 1202 基因
✓ CCLE oncokb_only: 417 基因
✓ CCLE protein: 19215 基因

STEP 3: 處理 TCGA 表達資料
✓ 使用欄位: sample_id
✓ TCGA 總基因數: 59427
✓ TCGA oncokb_all: 1208 基因
✓ TCGA oncokb_only: 418 基因
✓ 選擇了 1175 位患者的主要樣本

STEP 4: 處理註釋資料
✓ CCLE 可用藥物數: 31
✓ T

處理藥物: 100%|██████████| 31/31 [00:02<00:00, 13.72it/s]



✓ 完成 1267 筆預測
  跳過統計:
    - 細胞株數不足: 0
    - 分組樣本不足: 0
    - 有效基因不足: 0
  儲存至: ./tcga_drug_sensitivity_v2_all_drugs/quantile_30_oncokb_all.tsv

策略: quantile_20_protein


處理藥物: 100%|██████████| 31/31 [02:15<00:00,  4.36s/it]



✓ 完成 1266 筆預測
  跳過統計:
    - 細胞株數不足: 0
    - 分組樣本不足: 1
    - 有效基因不足: 0
  儲存至: ./tcga_drug_sensitivity_v2_all_drugs/quantile_20_protein.tsv

策略: spearman_top20_oncokb


處理藥物: 100%|██████████| 31/31 [00:10<00:00,  2.97it/s]


✓ 完成 1267 筆預測
  跳過統計:
    - 細胞株數不足: 0
    - 分組樣本不足: 0
    - 有效基因不足: 0
  儲存至: ./tcga_drug_sensitivity_v2_all_drugs/spearman_top20_oncokb.tsv

STEP 6: 集成多策略結果

策略覆蓋率分析:
2        1
3     1198
6       30
12       2
Name: count, dtype: int64

✅ 完成！最終統計
總預測數: 3,800
集成預測數（全部）: 1,232
集成預測數（>=2策略）: 1,232

策略分布:
strategy
quantile_30_oncokb_all    1267
spearman_top20_oncokb     1267
quantile_20_protein       1266
Name: count, dtype: int64

癌症類型分布:
cancer
Breast invasive carcinoma (BRCA)                278
Stomach adenocarcinoma (STAD)                   103
Bladder Urothelial Carcinoma (BLCA)              99
Sarcoma (SARC)                                   80
Pancreatic adenocarcinoma (PAAD)                 76
Uterine Corpus Endometrial Carcinoma (UCEC)      74
Uterine Carcinosarcoma (UCS)                     72
Lung adenocarcinoma (LUAD)                       66
Testicular Germ Cell Tumors (TGCT)               55
Head and Neck squamous cell carcinoma (HNSC)     48
Name: count, dtype: int64

輸出檔案